In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import *
from torchvision import datasets
from torchvision.transforms import *

In [ ]:
# Download ISTSat1 attitude dataset

DATAFILE1 = 'data/istsat1/NoSensorFault.csv'
DATAFILE2 = 'data/istsat1/SensorFault20000_25000.csv'

raw = np.loadtxt(DATAFILE2, delimiter=',')
print(f"Loaded {len(raw)} telemetry datapoints")

plt.figure()
for i in range(4):
    plt.plot(raw[:,1+i])
plt.show()

In [ ]:
WINDOW = 60
STRIDE = 1

class ISTSat1Dataset(Dataset):
    
    def __init__(self, raw_data):
        self.data = torch.tensor(raw_data, dtype=torch.float32)
        self.total = self.data.shape[0]
        self.size = 1 + int((self.total - WINDOW)/STRIDE)

    def __len__(self):
        return self.size

    def __getitem__(self, i):
        a = i * STRIDE
        b = a + WINDOW
        return self.data[a:b,1:5]

train_dataset = ISTSat1Dataset(raw[4000:])
print(f"Using {len(train_dataset)} windows for training.")

test_dataset = ISTSat1Dataset(raw[400:4000])
print(f"Using {len(test_dataset)} windows for testing.")
# Fault should be 1600:2100 in raw data index
# that is the same in windows (stride=1)


In [ ]:
# Show some examples of the dataset

(fig,ax) = plt.subplots(2, 5, constrained_layout=True)
for (i,d) in enumerate([train_dataset, test_dataset]):
    for j in range(5):
        r = np.random.randint(len(d))
        for k in range(4):
            ax[i,j].plot(d[r][:,k])
        ax[i,j].set_axis_off()
plt.show()

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X in train_dataloader:
    print(f"Shape of X: {X.shape}")
    break

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
class Autoencoder1(nn.Module):
    bottleneck = 128
    
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.unflatten = nn.Unflatten(1, (WINDOW,4))
        self.encoder = nn.Sequential(
            nn.Linear(WINDOW*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, self.bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.bottleneck, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, WINDOW*4),
        )

    def preprocess(self, x):
        return x
    
    def encode(self, x):
        x = self.flatten(x)
        coded = self.encoder(x)
        return coded

    def decode(self, x):
        x = self.decoder(x)
        decoded = self.unflatten(x)
        return decoded

    def forward(self, x):
        return self.decode( self.encode(x) )

In [ ]:
class Autoencoder2(nn.Module):
    bottleneck = 128
    
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv1d(4, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*WINDOW, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, self.bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.bottleneck, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 32*WINDOW//4),
            nn.ReLU(),
            nn.Unflatten(1, (32,WINDOW//4)),
            nn.ConvTranspose1d(32, 16, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose1d(16, 4, kernel_size=2, stride=2),
        )

    def preprocess(self, x):
        return x
    
    def encode(self, x):
        x = torch.permute(x, (0,2,1))
        coded = self.encoder(x)
        return coded

    def decode(self, x):
        y = self.decoder(x)
        return torch.permute(y, (0,2,1))

    def forward(self, x):
        return self.decode( self.encode(x) )

In [ ]:
# TO BE UPDATED

class ConvAutoencoderMNIST(nn.Module):
    bottleneck = 10
    
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=2, stride=2),
            nn.Sigmoid() # Output should be in the range [0, 1]
        )

    def encode(self, x):
        coded = self.encoder(x)
        return coded

    def decode(self, x):
        decoded = self.decoder(x)
        return decoded

    def forward(self, x):
        return self.decode( self.encode(x) )

In [ ]:
model = Autoencoder1().to(device)
print(model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, X in enumerate(dataloader):
        X = X.to(device)

        # Compute prediction error
        Z = model.preprocess(X)
        pred = model(Z)
        loss = loss_fn(pred, Z)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

In [ ]:
# Use this cell to save the model to a file
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
# Use this cell to load a model from a file
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

In [ ]:
# Show some examples of the dataset

model.eval()
(fig,ax) = plt.subplots(3, 5, constrained_layout=True)
ds = train_dataset
for j in range(5):
    r = np.random.randint(len(ds))
    x = ds[r]
    x = x[None,:].to(device)
    with torch.no_grad():
        y = model(x)
        # normalization to unit quaternions
        n = torch.sqrt(torch.sum(y**2, dim=2))
        y = y / n[:,:,None]

    for k in range(4):
        ax[0,j].plot(ds[r][:,k])
    ax[0,j].set_axis_off()
    ax[1,j].imshow(z[None,0].cpu(), cmap="gray")
    ax[1,j].set_axis_off()
    y = y.cpu()
    for k in range(4):
        ax[2,j].plot(y[0,:,k])
    ax[2,j].set_axis_off()
    
plt.axis('off')
plt.show()

In [ ]:
ds = test_dataset

all_data = torch.stack([ds[i] for i in range(len(ds))])
all_data_d = all_data.to(device)

model.eval()
with torch.no_grad():
    y = model(all_data_d)
    # normalization to unit quaternions
    n = torch.sqrt(torch.sum(y**2, dim=2))
    y = y / n[:,:,None]
    y = y.cpu()

rlosses = torch.sum((y - all_data)**2, dim=(1,2))

plt.figure()
plt.plot(rlosses)
plt.show()


In [ ]:
class Autoencoder3(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv1d(4, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*WINDOW, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )

        self.encoder = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
        )

        self.decoder = nn.Sequential(
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 256)
        )

    def preprocess(self, x):
        x = torch.permute(x, (0,2,1))
        y = self.cnn(x)
        return y
    
    def encode(self, x):
        coded = self.encoder(x)
        return coded

    def decode(self, x):
        decoded = self.decoder(x)
        return decoded

    def forward(self, x):
        return self.decode( self.encode(x) )

In [ ]:
model = Autoencoder3().to(device)
print(model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

In [ ]:
ds = test_dataset

all_data = torch.stack([ds[i] for i in range(len(ds))])
all_data_d = all_data.to(device)

model.eval()
with torch.no_grad():
    z = model.preprocess(all_data_d)
    y = model.decode(model.encode(z))
    y = y.cpu()
    z = z.cpu()

rlosses = torch.sum((y - z)**2, dim=1)

plt.figure()
plt.plot(rlosses)
plt.show()
